In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1909281,2021-12-05,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1909282,2021-12-06,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1909283,2021-12-07,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1909284,2021-12-08,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
49511,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
49513,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
49515,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
49517,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
49519,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
1160524,2021-12-05,Arkansas,Arkansas,5001,3545,60.00,5000,Arkansas,AR,Arkansas,State,3017804
1160526,2021-12-06,Arkansas,Arkansas,5001,3547,61.00,5000,Arkansas,AR,Arkansas,State,3017804
1160528,2021-12-07,Arkansas,Arkansas,5001,3549,61.00,5000,Arkansas,AR,Arkansas,State,3017804
1160530,2021-12-08,Arkansas,Arkansas,5001,3553,62.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1909281,2021-12-05,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1909282,2021-12-06,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1909283,2021-12-07,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1909284,2021-12-08,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-12-09') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
688,2021-12-09,Snohomish,Washington,73955,812.00,53061,WA,County,822083,8996.05,98.77
1374,2021-12-09,Cook,Illinois,697164,12160.00,17031,IL,County,5150233,13536.55,236.11
2059,2021-12-09,Orange,California,337623,5806.00,6059,CA,County,3175692,10631.48,182.83
2743,2021-12-09,Maricopa,Arizona,821487,13043.00,4013,AZ,County,4485414,18314.63,290.79
3427,2021-12-09,Los Angeles,California,1542590,27288.00,6037,CA,County,10039107,15365.81,271.82
...,...,...,...,...,...,...,...,...,...,...,...
1907717,2021-12-09,Wheeler,Oregon,120,1.00,41069,OR,County,1332,9009.01,75.08
1908140,2021-12-09,King,Texas,26,0.00,48269,TX,County,272,9558.82,0.00
1908532,2021-12-09,Esmeralda,Nevada,62,2.00,32009,NV,County,873,7101.95,229.10
1908920,2021-12-09,Loving,Texas,12,0.00,48301,TX,County,169,7100.59,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
688,2021-12-09,Snohomish,Washington,73955,812.00,53061,WA,County,822083,8996.05,98.77,98.77,8996.05
1374,2021-12-09,Cook,Illinois,697164,12160.00,17031,IL,County,5150233,13536.55,236.11,236.11,13536.55
2059,2021-12-09,Orange,California,337623,5806.00,6059,CA,County,3175692,10631.48,182.83,182.83,10631.48
2743,2021-12-09,Maricopa,Arizona,821487,13043.00,4013,AZ,County,4485414,18314.63,290.79,290.79,18314.63
3427,2021-12-09,Los Angeles,California,1542590,27288.00,6037,CA,County,10039107,15365.81,271.82,271.82,15365.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1907717,2021-12-09,Wheeler,Oregon,120,1.00,41069,OR,County,1332,9009.01,75.08,75.08,9009.01
1908140,2021-12-09,King,Texas,26,0.00,48269,TX,County,272,9558.82,0.00,0.00,9558.82
1908532,2021-12-09,Esmeralda,Nevada,62,2.00,32009,NV,County,873,7101.95,229.10,229.10,7101.95
1908920,2021-12-09,Loving,Texas,12,0.00,48301,TX,County,169,7100.59,0.00,0.00,7100.59


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
1213935,2021-12-09,Hale,Alabama,3210,89.00,1065,AL,County,14651,21909.77,607.47,607.47,21909.77,1
1322355,2021-12-09,Clarke,Alabama,4864,86.00,1025,AL,County,23622,20590.97,364.07,364.07,20590.97,2
1213315,2021-12-09,Fayette,Alabama,3333,85.00,1057,AL,County,16302,20445.34,521.41,521.41,20445.34,3
704046,2021-12-09,Franklin,Alabama,6398,108.00,1059,AL,County,31362,20400.48,344.37,344.37,20400.48,4
1024204,2021-12-09,Winston,Alabama,4813,111.00,1133,AL,County,23629,20369.04,469.76,469.76,20369.04,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1158047,2021-12-09,Converse,Wyoming,2352,45.00,56009,WY,County,13822,17016.35,325.57,325.57,17016.35,19
1021089,2021-12-09,Johnson,Wyoming,1373,16.00,56019,WY,County,8445,16258.14,189.46,189.46,16258.14,20
1158669,2021-12-09,Sublette,Wyoming,1502,23.00,56035,WY,County,9831,15278.20,233.95,233.95,15278.20,21
1442672,2021-12-09,Lincoln,Wyoming,3001,25.00,56023,WY,County,19830,15133.64,126.07,126.07,15133.64,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
948081,2021-12-09,Lowndes,Alabama,1783,68.00,1085,AL,County,9726,18332.31,699.16,699.16,18332.31,27,1
1213935,2021-12-09,Hale,Alabama,3210,89.00,1065,AL,County,14651,21909.77,607.47,607.47,21909.77,1,2
401217,2021-12-09,Walker,Alabama,12245,380.00,1127,AL,County,63521,19277.09,598.23,598.23,19277.09,13,3
946833,2021-12-09,Crenshaw,Alabama,2626,78.00,1041,AL,County,13772,19067.67,566.37,566.37,19067.67,15,4
839337,2021-12-09,Dallas,Alabama,5320,210.00,1047,AL,County,37196,14302.61,564.58,564.58,14302.61,62,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1021089,2021-12-09,Johnson,Wyoming,1373,16.00,56019,WY,County,8445,16258.14,189.46,189.46,16258.14,20,19
1406328,2021-12-09,Uinta,Wyoming,4084,31.00,56041,WY,County,20226,20191.83,153.27,153.27,20191.83,10,20
1442672,2021-12-09,Lincoln,Wyoming,3001,25.00,56023,WY,County,19830,15133.64,126.07,126.07,15133.64,22,21
944337,2021-12-09,Albany,Wyoming,6885,40.00,56001,WY,County,38880,17708.33,102.88,102.88,17708.33,17,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,98.77,8996.05,31,28
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,98.77,8996.05,31,28
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,98.77,8996.05,31,28
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,98.77,8996.05,31,28
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,98.77,8996.05,31,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1904864,2021-12-05,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1904865,2021-12-06,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1904866,2021-12-07,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1904867,2021-12-08,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa[ '14day_case_avg' ] = usa.case_day.rolling(14).mean()
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg
1208899,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,607.47,21909.77,2,1,1.00,nan
1208900,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,607.47,21909.77,2,1,0.00,nan
1208901,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,607.47,21909.77,2,1,0.00,nan
1208902,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,607.47,21909.77,2,1,0.00,nan
1208903,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,607.47,21909.77,2,1,2.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1467156,2021-12-05,Crook,Wyoming,1000,21.00,56011,WY,County,7584,13185.65,276.90,276.90,13238.40,11,23,0.00,1.29
1467157,2021-12-06,Crook,Wyoming,1002,21.00,56011,WY,County,7584,13212.03,276.90,276.90,13238.40,11,23,2.00,1.36
1467158,2021-12-07,Crook,Wyoming,1004,21.00,56011,WY,County,7584,13238.40,276.90,276.90,13238.40,11,23,2.00,1.43
1467159,2021-12-08,Crook,Wyoming,1004,21.00,56011,WY,County,7584,13238.40,276.90,276.90,13238.40,11,23,0.00,1.43


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa[ '14day_death_avg' ] = usa.death_day.rolling(14).mean()
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
944285,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,699.16,18332.31,1,27,1.00,12.43,0.00,nan
944286,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,699.16,18332.31,1,27,0.00,11.71,0.00,nan
944287,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,699.16,18332.31,1,27,0.00,11.43,0.00,nan
944288,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,699.16,18332.31,1,27,0.00,11.36,0.00,nan
944289,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,699.16,18332.31,1,27,0.00,10.43,0.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396772,2021-12-05,Teton,Wyoming,5406,14.00,56039,WY,County,23464,23039.55,59.67,59.67,23167.41,23,3,0.00,4.00,0.00,0.00
396773,2021-12-06,Teton,Wyoming,5417,14.00,56039,WY,County,23464,23086.43,59.67,59.67,23167.41,23,3,11.00,3.93,0.00,0.00
396774,2021-12-07,Teton,Wyoming,5425,14.00,56039,WY,County,23464,23120.53,59.67,59.67,23167.41,23,3,8.00,4.21,0.00,0.00
396775,2021-12-08,Teton,Wyoming,5433,14.00,56039,WY,County,23464,23154.62,59.67,59.67,23167.41,23,3,8.00,4.79,0.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-12-09') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
479429,2021-12-09,Imperial,California,39364,781.00,6025,CA,County,181215,21722.26,430.98,430.98,21722.26,1,3,136.00,88.29,4.00,0.93
237845,2021-12-09,San Bernardino,California,379756,6006.00,6071,CA,County,2180085,17419.32,275.49,275.49,17419.32,2,6,649.00,562.43,-1.00,4.43
3427,2021-12-09,Los Angeles,California,1542590,27288.00,6037,CA,County,10039107,15365.81,271.82,271.82,15365.81,3,15,1686.00,1381.86,13.00,15.29
1159220,2021-12-09,Tuolumne,California,7979,146.00,6109,CA,County,54478,14646.28,268.00,268.00,14646.28,4,19,14.00,12.29,1.00,0.36
105933,2021-12-09,Stanislaus,California,92542,1449.00,6099,CA,County,550660,16805.65,263.14,263.14,16805.65,5,8,122.00,105.57,8.00,2.57
73279,2021-12-09,Shasta,California,26428,470.00,6089,CA,County,180080,14675.70,261.00,261.00,14675.70,6,18,31.00,36.50,5.00,2.14
842414,2021-12-09,Inyo,California,2605,46.00,6027,CA,County,18039,14440.93,255.00,255.00,14440.93,7,20,3.00,4.43,0.00,0.07
762929,2021-12-09,Merced,California,46013,684.00,6047,CA,County,277680,16570.51,246.33,246.33,16570.51,8,9,52.00,57.57,0.00,1.00
1022900,2021-12-09,Kings,California,35671,372.00,6031,CA,County,152940,23323.53,243.23,243.23,23323.53,9,2,43.00,45.21,0.00,0.64
84175,2021-12-09,San Joaquin,California,108740,1841.00,6077,CA,County,762148,14267.57,241.55,241.55,14267.57,10,22,150.00,124.29,2.00,0.29


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
1798882,2021-12-09,Lassen,California,8019,61.00,6035,CA,County,30573,26229.03,199.52,199.52,26229.03,17,1,7.00,6.86,0.00,0.29
1022900,2021-12-09,Kings,California,35671,372.00,6031,CA,County,152940,23323.53,243.23,243.23,23323.53,9,2,43.00,45.21,0.00,0.64
479429,2021-12-09,Imperial,California,39364,781.00,6025,CA,County,181215,21722.26,430.98,430.98,21722.26,1,3,136.00,88.29,4.00,0.93
131490,2021-12-09,Tulare,California,87470,1102.00,6107,CA,County,466195,18762.53,236.38,236.38,18762.53,11,4,334.00,132.57,-1.00,1.43
299385,2021-12-09,Kern,California,159493,1862.00,6029,CA,County,900202,17717.47,206.84,206.84,17717.47,14,5,157.00,212.57,7.00,5.79
237845,2021-12-09,San Bernardino,California,379756,6006.00,6071,CA,County,2180085,17419.32,275.49,275.49,17419.32,2,6,649.00,562.43,-1.00,4.43
48224,2021-12-09,Madera,California,26645,315.00,6039,CA,County,157327,16936.06,200.22,200.22,16936.06,16,7,78.00,45.71,4.00,1.07
105933,2021-12-09,Stanislaus,California,92542,1449.00,6099,CA,County,550660,16805.65,263.14,263.14,16805.65,5,8,122.00,105.57,8.00,2.57
762929,2021-12-09,Merced,California,46013,684.00,6047,CA,County,277680,16570.51,246.33,246.33,16570.51,8,9,52.00,57.57,0.00,1.00
1093295,2021-12-09,Glenn,California,4621,51.00,6021,CA,County,28393,16275.14,179.62,179.62,16275.14,21,10,16.00,5.00,0.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
479429,2021-12-09,Imperial,California,39364,781.00,6025,CA,County,181215,21722.26,430.98,430.98,21722.26,1,3,136.00,88.29,4.00,0.93
237845,2021-12-09,San Bernardino,California,379756,6006.00,6071,CA,County,2180085,17419.32,275.49,275.49,17419.32,2,6,649.00,562.43,-1.00,4.43
3427,2021-12-09,Los Angeles,California,1542590,27288.00,6037,CA,County,10039107,15365.81,271.82,271.82,15365.81,3,15,1686.00,1381.86,13.00,15.29
1159220,2021-12-09,Tuolumne,California,7979,146.00,6109,CA,County,54478,14646.28,268.00,268.00,14646.28,4,19,14.00,12.29,1.00,0.36
105933,2021-12-09,Stanislaus,California,92542,1449.00,6099,CA,County,550660,16805.65,263.14,263.14,16805.65,5,8,122.00,105.57,8.00,2.57
73279,2021-12-09,Shasta,California,26428,470.00,6089,CA,County,180080,14675.70,261.00,261.00,14675.70,6,18,31.00,36.50,5.00,2.14
842414,2021-12-09,Inyo,California,2605,46.00,6027,CA,County,18039,14440.93,255.00,255.00,14440.93,7,20,3.00,4.43,0.00,0.07
762929,2021-12-09,Merced,California,46013,684.00,6047,CA,County,277680,16570.51,246.33,246.33,16570.51,8,9,52.00,57.57,0.00,1.00
1022900,2021-12-09,Kings,California,35671,372.00,6031,CA,County,152940,23323.53,243.23,243.23,23323.53,9,2,43.00,45.21,0.00,0.64
84175,2021-12-09,San Joaquin,California,108740,1841.00,6077,CA,County,762148,14267.57,241.55,241.55,14267.57,10,22,150.00,124.29,2.00,0.29


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'current_crate', 'state_case', 'case_day', '14day_case_avg']]
top10_ca_case_df

,date,county,cases,current_crate,state_case,case_day,14day_case_avg
0,03/20/20,Imperial,4,2.21,3,4.00,45.50
1,03/21/20,Imperial,4,2.21,3,0.00,45.50
2,03/22/20,Imperial,4,2.21,3,0.00,45.50
3,03/23/20,Imperial,4,2.21,3,0.00,33.64
4,03/24/20,Imperial,9,4.97,3,5.00,33.86
...,...,...,...,...,...,...,...
6250,12/05/21,Glenn,4585,16148.35,10,0.00,2.86
6251,12/06/21,Glenn,4603,16211.74,10,18.00,3.93
6252,12/07/21,Glenn,4603,16211.74,10,0.00,3.86
6253,12/08/21,Glenn,4605,16218.79,10,2.00,3.86


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','deaths', 'current_drate', 'state_death', 'death_day', '14day_death_avg']]
top10_ca_death_df

,date,county,deaths,current_drate,state_death,death_day,14day_death_avg
0,03/20/20,Imperial,0.00,0.00,1,0.00,0.00
1,03/21/20,Imperial,0.00,0.00,1,0.00,0.00
2,03/22/20,Imperial,0.00,0.00,1,0.00,0.00
3,03/23/20,Imperial,0.00,0.00,1,0.00,0.00
4,03/24/20,Imperial,0.00,0.00,1,0.00,0.00
...,...,...,...,...,...,...,...
6359,12/05/21,San Joaquin,1839.00,241.29,10,0.00,0.29
6360,12/06/21,San Joaquin,1839.00,241.29,10,0.00,0.14
6361,12/07/21,San Joaquin,1839.00,241.29,10,0.00,0.14
6362,12/08/21,San Joaquin,1839.00,241.29,10,0.00,0.14


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)